In [1]:
import numpy as np
!pip install import-ipynb
import import_ipynb
!pip install ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=5389dc478f5f6e2ab727190d96fc6791eebc661ccd20dfd0742a57814eb5414c
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [2]:
import torch
import json
import nltk
nltk.download('punkt')

import nltk
#from nltk_utils import get_ipython
from nltk.stem.porter import PorterStemmer
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
stemmer=PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stemming(word):
    return stemmer.stem(word.lower())

def bagofwords(tokenized_sentence,all_words):
    tokenized_sentence=[stemming(w) for w in tokenized_sentence]
    bag=np.zeros(len(all_words),dtype=np.float32)
    for index,word in enumerate(all_words):
        if word in tokenized_sentence:
            bag[index]=1.0
    return bag        
    
    


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
with open("intents1.json","r") as f:
    intents=json.load(f)

In [4]:
all_words=[]
tags=[]
xy=[]
for intent in intents["intents"]:
    tags.append(intent["tag"])
    for sentence in intent["patterns"]:
        words=tokenize(sentence)
        all_words.extend(words)
        xy.append((words,intent["tag"]))

In [5]:
ignore_words=["?",".",",","!"]

In [6]:
all_words=[stemming(word) for word in all_words if word not in ignore_words]
len(all_words)

88

In [7]:
all_words=sorted(set(all_words))

In [8]:
tags=sorted(tags)
tags

['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']

In [9]:
X_train=[]
y_train=[]
for (sentence,tag) in xy:
    bag=bagofwords(sentence,all_words)
    X_train.append(bag)
    label=tags.index(tag)
    y_train.append(label)
    
X_train=np.array(X_train)
y_train=np.array(y_train)

In [10]:
X_train
y_train

array([3, 3, 3, 3, 3, 3, 2, 2, 2, 6, 6, 6, 6, 4, 4, 4, 5, 5, 5, 5, 0, 0,
       0, 1, 1, 1])

In [11]:
class BotDataset:
    def __init__(self):
        self.num_samples=len(X_train)
        self.x_data=X_train
        self.y_data=y_train
        
    def __getitem__(self,index):
        return self.x_data[index],self.y_data[index]
    
    def __len__(self):
        return self.num_samples

In [26]:
dataset=BotDataset()
train_loader=DataLoader(dataset=dataset,batch_size=4,shuffle=True,num_workers=2)

### Model Bulding

In [27]:
class NeuralNet(nn.Module):
    def __init__(self,input_size,size_l1,size_l2,size_l3,num_classes):
        super().__init__()
        self.l1=nn.Linear(input_size,size_l1)
        self.l2=nn.Linear(size_l1,size_l2)
        self.l3=nn.Linear(size_l2,size_l3)
        self.l4=nn.Linear(size_l3,num_classes)
        self.relu=nn.ReLU()
        
    def forward(self,x):
        output=self.l1(x)
        output=self.relu(output)
        output=self.l2(output)
        output=self.relu(output)
        output=self.l3(output)
        output=self.relu(output)
        output=self.l4(output)
        return output
        
            
        

In [28]:
model=NeuralNet(len(all_words),10,20,10,len(tags))

In [29]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)
num_epochs=400
train_loader

In [30]:
for epoch in range(num_epochs):
    for (words,label) in train_loader:
        pred=model.forward(words)
        loss=criterion(pred,label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if epoch%10==0:
        print("epoch: {} and loss: {}".format(epoch,loss.item()))
    
    

epoch: 0 and loss: 1.749819040298462
epoch: 10 and loss: 1.7998692989349365
epoch: 20 and loss: 2.1232807636260986
epoch: 30 and loss: 1.4061774015426636
epoch: 40 and loss: 1.4614498615264893
epoch: 50 and loss: 0.6012253165245056
epoch: 60 and loss: 0.2649418115615845
epoch: 70 and loss: 0.08440493047237396
epoch: 80 and loss: 0.13002842664718628
epoch: 90 and loss: 0.009104873053729534
epoch: 100 and loss: 0.0523400716483593
epoch: 110 and loss: 0.021827595308423042
epoch: 120 and loss: 0.008811363950371742
epoch: 130 and loss: 0.006072648335248232
epoch: 140 and loss: 0.0017492328770458698
epoch: 150 and loss: 0.010148456320166588
epoch: 160 and loss: 0.006226967088878155
epoch: 170 and loss: 0.0017255402635782957
epoch: 180 and loss: 0.0016604505944997072
epoch: 190 and loss: 0.0034376392140984535
epoch: 200 and loss: 0.003710213117301464
epoch: 210 and loss: 0.0029225756879895926
epoch: 220 and loss: 0.0022174331825226545
epoch: 230 and loss: 0.000758479000069201
epoch: 240 and l

### Implementing Chat Bot 

In [31]:
model.eval()

NeuralNet(
  (l1): Linear(in_features=54, out_features=10, bias=True)
  (l2): Linear(in_features=10, out_features=20, bias=True)
  (l3): Linear(in_features=20, out_features=10, bias=True)
  (l4): Linear(in_features=10, out_features=7, bias=True)
  (relu): ReLU()
)

In [32]:
model
import random

In [33]:
bot_name = "Saket"
print("Let's chat! (type 'q' to exit)")
while True:

    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "q":
        break

    sentence = tokenize(sentence)
    X = bagofwords(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.5:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print("{} :{}".format(bot_name,random.choice(intent["responses"])))
    else:
        print(f"{bot_name}: I do not understand...")

Let's chat! (type 'q' to exit)
You: Hey
Saket :Hey :-)
You: what do you sell?
Saket :We sell coffee and tea
You: I want to order
Saket :My pleasure
You: i want to order tea
Saket :Happy to help!
You: how can I order?
Saket :Any time!
You: do you accept cash?
Saket :We accept most major credit cards, and Paypal
You: i will pay by credit card
Saket :We accept most major credit cards, and Paypal
You: okay
Saket: I do not understand...
You: thank you
Saket :Any time!
You: see you
Saket :Have a nice day
You: bye
Saket :Bye! Come back again soon.
You: take care
Saket :My pleasure
You: q


In [ ]:
hi
